In [2]:
import requests
from scrapy.http import TextResponse
import re
import pandas as pd
import time
from tqdm import trange, tqdm
from pprint import pprint

In [3]:
def get_response(url):
    r = requests.get(url)
    return TextResponse(r.url, body=r.text, encoding='utf-8')

def get_raw_url(url):
    return re.search(r'https?.*\.html', url).group(0)

def discard_xml_markers(string):
    return re.sub(r'<(.*?)>', ' ', string)

def squash_whitespaces(string):
    return re.sub(r'\s+', ' ', string)

def get_main(house):
    main = house.xpath('//section[@class="section-offer-params"]//ul[@class="main-list"]/li').extract()
    main = [discard_xml_markers(param) for param in main]
    return main

def get_sub(house):
    sub = house.xpath('//section[@class="section-offer-params"]//ul[@class="sub-list"]/li').extract()
    sub = [discard_xml_markers(param) for param in sub]
    return sub

def get_dotted(house):
    dotted = house.xpath('//section[@class="section-offer-params"]//ul[@class="dotted-list"]/li').extract()
    dotted = [discard_xml_markers(param) for param in dotted]
    return dotted

def get_place(house):
    place = house.xpath('//p[@class="address-text" and @itemprop]/text()').extract()
    return place

def get_coords(house):
    coords = house.xpath('//section[@class="section-offer-map hidden-print"]//div[@id="adDetailInlineMap"]')
    lat = coords.xpath('//@data-poi-lat').extract()
    lon = coords.xpath('//@data-poi-lon').extract()
    return (lat, lon)

def get_meta(house):
    meta = house.xpath('//div[@class="text-details clearfix"]').extract()[0]
    return squash_whitespaces(discard_xml_markers(meta))

In [5]:
all_offers_template = 'https://www.otodom.pl/sprzedaz/mieszkanie/krakow/?search%5Bdist%5D=15&nrAdsPerPage=72&page={}'
# all_offers_template = 'https://www.otodom.pl/sprzedaz/mieszkanie/?search%5Bdescription%5D=1&nrAdsPerPage=72&page={}'

In [6]:
for page in trange(1, 420):
    response = get_response(all_offers_template.format(page))
    
    houses = response.xpath('//article[@data-url]/@data-url').extract()
    houses = [get_raw_url(url) for url in houses]
    pprint(houses)
    data = []
    for house_url in tqdm(houses):
#         print(house_url)
        house = get_response(house_url)
#         time.sleep(0.1)
        data.append({
            'main': get_main(house),
            'sub': get_sub(house),
            'dotted': get_dotted(house),
            'place': get_place(house),
            'coords': get_coords(house),
            'meta': get_meta(house)
            
        })
        pprint(data)
        break
    break
#     pd.DataFrame(data).to_csv('./data/data_part_{}.csv'.format(page), index=False)

  0%|          | 0/79 [00:00<?, ?it/s]

['https://www.otodom.pl/malopolskie/oferta/poszukiwana-lokalizacja-3pok-20-min-od-centrum-ID3g9Ve.html',
 'https://www.otodom.pl/malopolskie/oferta/spelniaj-marzenia-nie-baczac-na-cene-angel-city-ID3fCGg.html',
 'https://www.otodom.pl/malopolskie/oferta/k-wielkiego-krowodrza-gotowe-do-zamieszkania-ID3fTkg.html',
 'https://www.otodom.pl/malopolskie/oferta/nowe-3-pokojowe-mieszkanie-o-powierzchni-43-m2-ID360Xe.html',
 'https://www.otodom.pl/malopolskie/oferta/mieszkanie-4pok-87m2-garaz-ogrod-kom-lok-krakow-ID3aQbY.html',
 'https://www.otodom.pl/malopolskie/oferta/mieszkanie-w-stanie-idealnym-ID3f1Zm.html',
 'https://www.otodom.pl/malopolskie/oferta/swietna-lokalizacja-ruczaj-2-pokoje-od-zaraz-ID3dCZV.html',
 'https://www.otodom.pl/oferta/mieszkanie-43-52-m-krakow-ID3gyeS.html',
 'https://www.otodom.pl/oferta/2-pok-z-duzym-balkonem-o-pow-55m2-dobrego-pasterza-ID3gyeY.html',
 'https://www.otodom.pl/oferta/apartament-na-starym-miescie-43m2-pod-inwestycje-ID3gyeQ.html',
 'https://www.otodom.

[{'coords': (['50.09467840'], ['19.96935590']),
  'dotted': [' domofon / wideofon ',
             ' meble ',
             ' balkon ',
             '  piwnica ',
             '  winda ',
             '  oddzielna kuchnia ',
             '  pom. użytkowe '],
  'main': [' cena   320 000 zł   6 136,15 zł/m² ',
           ' powierzchnia   52,15 m²   ',
           ' liczba pokoi   3   ',
           ' piętro   8  (z 12)  '],
  'meta': ' Nr oferty w Otodom: 48180462 Nr oferty w biurze nieruchomości: '
          'ABN-MS-21827 Liczba wyświetleń strony: 3398 Data dodania: '
          '10.05.2017 Data aktualizacji: 17.05.2017 ',
  'place': ['Kraków, Prądnik Czerwony, Powstańców'],
  'sub': ['  rynek:  wtórny ',
          '  rodzaj zabudowy:  blok ',
          '  okna:  plastikowe ',
          '  ogrzewanie:  miejskie ',
          '  rok budowy:  1980  ',
          '  stan wykończenia:  do zamieszkania ',
          '  czynsz:  350 zł ',
          '  forma własności:  pełna własność ']}]


In [1]:
!jupyter nbconvert --to script scraping_sandbox.ipynb


[NbConvertApp] Converting notebook scraping_sandbox.ipynb to script
[NbConvertApp] Writing 2616 bytes to scraping_sandbox.py
